In [1]:
from src.convrnn_main import *
import os

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

In [4]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [ ]:
config = {"output_path" : "/home/jl5307/current_research/AMD_prediction/results/convlstm/",
        "data_path" : "/home/jl5307/current_research/AMD_prediction/img_data/numpy_data/",
        "architecture" : [3, 4, 23, 3], "epoch" : 20, "per_replica_batch_size" : 4, "units" : 256, "learning_rate" : 0.0005, 
          "designate_gpu" : ["0","1","2","3"], "l2_reg" : 0.0001, "class_weight" : [0.1, 0.9], "lr_scheduling": True,
          "use_pretrain" : "imagenet", "pretrained_weights_path" : "/home/jl5307/current_research/AMD_prediction/results/resnet101_binary/b16_lr0005_e20_detection_imagenet/resnet_best_model.npy", 
          "pretrained_config_path" : "/home/jl5307/current_research/AMD_prediction/results/resnet101_binary/b16_lr0005_e20_detection_imagenet/resnet_config.pkl",
          "testing" : True}

In [5]:
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 2


In [1]:
from src.convrnn_distributed_main import *

In [1]:
from src.convrnn_distributed_main_ex import *

In [2]:
distributed_train_resnetlstm_module("/home/jl5307/current_research/AMD_prediction/results/convlstm/",
                                   "/home/jl5307/current_research/AMD_prediction/img_data/numpy_data/",
                                   [3, 4, 23, 3], 1, 4, 256, 0.0005, "0,1,2,3", 0.0001, [0.1, 0.9], lr_scheduling=True, use_pretrain="imagenet",
                                   pretrained_weights_path="/home/jl5307/current_research/AMD_prediction/results/resnet101_binary/b16_lr0005_e20_detection_imagenet/resnet_best_model.npy",
                                   pretrained_config_path="/home/jl5307/current_research/AMD_prediction/results/resnet101_binary/b16_lr0005_e20_detection_imagenet/resnet_config.pkl",
                                   testing=False)

set mirrored strategy using 4 GPUs...
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
load data...
build and initialize models...
use pretrained weights based on imagenet...
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:

KeyboardInterrupt: 

In [5]:
validation_dataset_path = os.path.join("/home/jl5307/current_research/AMD_prediction/img_data/numpy_data/", "validation_dataset_tf")

In [6]:
data_path = "/home/jl5307/current_research/AMD_prediction/img_data/numpy_data/"

In [7]:
global_batch_size = 4

In [14]:
validation_dataset_elem_spec_path = os.path.join(data_path, "validation_dataset_tf_element_spec.pkl")
validation_dataset_elem_spec = load_data(validation_dataset_elem_spec_path)
validation_dataset = tf.data.experimental.load(validation_dataset_path, element_spec=validation_dataset_elem_spec)
validation_dataset = validation_dataset.shuffle(len(validation_dataset)).batch(2)

In [15]:
len(validation_dataset)

580

In [11]:
inputs = iterator.get_next()

In [13]:
x_batch, y_batch = inputs

In [18]:
x_batch.shape[4]

3

In [7]:
for idx, d in enumerate(validation_dataset):
    if idx == 0:
        print(d)
        break

(<tf.Tensor: shape=(4, 12, 224, 224, 3), dtype=float64, numpy=
array([[[[[127., 127., 128.],
          [126., 130., 128.],
          [129., 129., 127.],
          ...,
          [130., 133., 131.],
          [128., 128., 128.],
          [135., 131., 131.]],

         [[130., 129., 129.],
          [130., 130., 126.],
          [130., 130., 130.],
          ...,
          [130., 130., 130.],
          [128., 128., 132.],
          [128., 128., 128.]],

         [[130., 129., 129.],
          [126., 126., 126.],
          [124., 124., 132.],
          ...,
          [128., 126., 128.],
          [131., 135., 130.],
          [128., 128., 128.]],

         ...,

         [[129., 124., 129.],
          [128., 128., 128.],
          [124., 128., 128.],
          ...,
          [124., 128., 128.],
          [120., 124., 124.],
          [125., 129., 129.]],

         [[120., 124., 131.],
          [129., 133., 133.],
          [124., 128., 128.],
          ...,
          [123., 126., 126.],

In [1]:
import tensorflow as tf
import numpy as np

In [26]:
class WeightedBinaryCrossEntropy(tf.keras.losses.Loss):
    def __init__(self, class_weight, **kwargs):
        super(WeightedBinaryCrossEntropy, self).__init__()
        self.class_weight = tf.cast(np.reshape(class_weight, newshape=(2,1)), tf.float32)
        self.eps = 10e-07 
        self.reduction = tf.keras.losses.Reduction.SUM
    
    def call(self, y_true, y_pred):  
        # Clipping y_pred for numerical stability 
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        y_pred = tf.reshape(y_pred, shape=(y_true.shape[0], y_true.shape[1])) # make the shape of y_true and y_pred the same
        y_pred = tf.clip_by_value(y_pred, self.eps, 1-self.eps)

        loss_for_true = tf.negative(tf.reshape(tf.multiply(y_true, tf.math.log(y_pred)), shape=(1,-1)) * self.class_weight[1])
        loss_for_false = tf.negative(tf.reshape(tf.multiply(1. - y_true, tf.math.log(1.0 - y_pred)), shape=(1,-1)) * self.class_weight[0])
        weighted_loss = tf.add(loss_for_true, loss_for_false)
        
        return weighted_loss

In [27]:
WBCE = WeightedBinaryCrossEntropy([0.1, 0.9])

In [28]:
WBCE.get_config()

{'reduction': 'sum', 'name': None}

In [29]:
yt = np.array([[0., 1.], [1., 0.]])

In [30]:
yp = np.array([[0.8, 0.2], [0.3, 0.7]])

In [31]:
WBCE(y_true=yt, y_pred=yp)

<tf.Tensor: shape=(), dtype=float32, numpy=2.8134108>

In [10]:
BCE = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)

In [11]:
BCE(yt, yp)

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([1.60943747, 1.20397246])>

In [3]:
load_data("/home/jl5307/current_research/AMD_prediction/results/convlstm/convrnn_result_dict.pkl")

{'entire_AUC': 0.7653896,
 1: 0.5,
 2: 0.2983529,
 3: 0.3214286,
 4: 0.31311545,
 5: 0.21054544,
 6: 0.24751067,
 7: 0.2871486,
 8: 0.31600547,
 9: 0.33509392,
 10: 0.35029548,
 11: 0.36036035,
 12: 0.0}